In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import b3d
import sys
sys.path.append(b3d.get_root_path())

In [3]:
from tests.patch_tracking.task import PatchTrackingTask
import rerun as rr

In [16]:
spec = PatchTrackingTask.get_known_unity_scene_specs()[0]
task = PatchTrackingTask.task_from_known_unity_scene_spec(spec, n_frames=5)


In [19]:
rr.init(f'task--{spec["scene_name"]}-2')
rr.connect("127.0.0.1:8812")
task.visualize_task()

In [21]:
task_spec = task.get_task_specification()

In [22]:
from tests.patch_tracking.solver import AdamPatchTracker
solver = AdamPatchTracker()
solution = solver.solve(task_spec)

In [23]:
solver.visualize_solver_state(task_spec)

/home/georgematheos/b3d/b3d/chisight/dense/model.py:124: RerunWarning: Pinhole: ValueError(Expected focal_length to be one or two floats)
  rr.Pinhole(
